In [24]:
import torch
from torch import nn
import torch.nn.functional as F

import gym
from pfrl.policies import SoftmaxCategoricalHead
from pfrl.agents import REINFORCE
from pfrl.replay_buffers import ReplayBuffer

from repaired.replay import ExperienceCollection

In [25]:
EPS = 1e-2
GAMMA = 0.9
EPSILON = 0.3
N_EPISODES = 100
MAX_EPISODE_LEN = 200

HIDDEN_SIZE = 200

In [26]:
env = gym.make('CartPole-v0')
obs_size = env.observation_space.low.size
n_actions = env.action_space.n

/Users/education/DATA/projects/ai/reinforcement-learning/repaired/venv/lib/python3.9/site-packages/gym/envs/registration.py:505: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1` with the environment ID `CartPole-v1`.
  logger.warn(


In [27]:
model = nn.Sequential(
    nn.Linear(obs_size, HIDDEN_SIZE),
    nn.LeakyReLU(0.2),
    nn.Linear(HIDDEN_SIZE, HIDDEN_SIZE),
    nn.LeakyReLU(0.2),
    nn.Linear(HIDDEN_SIZE, n_actions),
    SoftmaxCategoricalHead(),
)

optimizer = torch.optim.Adam(model.parameters(), eps=EPS)
agent = REINFORCE(model, optimizer)

In [44]:
levels = torch.tensor([1, 2, 3, 4, 5])
# score_levels = torch.zeros_like(levels) # global level scores
# last_count_levels = torch.zeros_like(levels) # global level timestamps
# last_episode = 0
# visited_levels = torch.tensor([])

In [29]:
last_episode = 0
replay_buffer = ReplayBuffer()

for i in range(1, N_EPISODES + 1):
    obs = env.reset()
    score_levels = torch.zeros_like(levels) # global level scores
    last_count_levels = torch.zeros_like(levels) # global level timestamps

    R = 0  # return (sum of rewards)
    t = 0  # time step
    while True:
        # Uncomment to watch the behavior in a GUI window
        # env.render()
        action = agent.act(obs)
        obs, reward, done, _ = env.step(action)
        R += reward
        t += 1
        reset = t == MAX_EPISODE_LEN
        agent.observe(obs, reward, done, reset)
        
        if done or reset:
            break
    if i % 10 == 0:
        print('episode:', i, 'R:', R)
    if i % 50 == 0:
        print('statistics:', agent.get_statistics())

    last_episode += 1

episode: 10 R: 63.0
episode: 20 R: 27.0
episode: 30 R: 48.0
episode: 40 R: 135.0
episode: 50 R: 146.0
statistics: [('average_entropy', 0.5440467726716249)]
episode: 60 R: 161.0
episode: 70 R: 105.0
episode: 80 R: 200.0
episode: 90 R: 78.0
episode: 100 R: 184.0
statistics: [('average_entropy', 0.5520932629083812)]
Finished.
